In [ ]:
!pip install transformers==4.40.1 # Use this version and Python 3.10 for stable compatibility
!pip install timeseriesviz

import torch
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from csv import reader, writer
from timeseriesviz import plot_numpy

In [ ]:
timeseries_prop = "prcp(mm/day)"          # Choose from prcp(mm/day), srad(W/m2), tmax(C), tmin(C), vp(Pa), QObs(mm/d)
ckpt_name = f"CamelsUS-sundial-prcp-ckpt"
run_name = f"CamelsUS-sundial-prcp"

Restorefromcheckpoint = False       # Restore from checkpoint

In [ ]:
def restoreValLocs(ValidationRunName):
    InputFileName = APPLDIR + 'Validation' + ValidationRunName
    with open(InputFileName, 'r', newline='') as inputfile:
        Myreader = reader(inputfile, delimiter=',')
        header = next(Myreader)
        LocationValidationFraction = np.float32(header[0])
        TrainingNloc = np.int32(header[1])
        ValidationNloc = np.int32(header[2])

        ListofTrainingLocs = np.empty(TrainingNloc, dtype = np.int32)
        ListofValidationLocs = np.empty(ValidationNloc,  dtype = np.int32)
        nextrow = next(Myreader)
        for iloc in range(0, TrainingNloc):
            ListofTrainingLocs[iloc] = np.int32(nextrow[iloc])
        nextrow = next(Myreader)
        for iloc in range(0, ValidationNloc):
            ListofValidationLocs[iloc] = np.int32(nextrow[iloc])


    return TrainingNloc, ValidationNloc, ListofTrainingLocs, ListofValidationLocs


def saveCkpt(iloc, ground_truth, preds, run_name, ckpt_name):
    OutputFileName = APPLDIR + ckpt_name
    with open(OutputFileName, 'w', newline='') as outputfile:
        Mywriter = writer(outputfile, delimiter=',')
        Mywriter.writerow([iloc])

    np.save(APPLDIR + f"{run_name}_yin.npy", ground_truth)
    np.save(APPLDIR + f"{run_name}_FitPredictions.npy", preds)


def restoreCkpt(run_name, ckpt_name):
    InputFileName = APPLDIR + ckpt_name
    with open(InputFileName, 'r', newline='') as inputfile:
        Myreader = reader(inputfile, delimiter=',')
        iloc = next(Myreader)[0]

    ground_truth = np.load(APPLDIR + f"{run_name}_yin.npy", allow_pickle=True)
    preds = np.load(APPLDIR + f"{run_name}_FitPredictions.npy", allow_pickle=True)

    return int(iloc), ground_truth, preds

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
APPLDIR = "/content/gdrive/My Drive/Colab Datasets/Hydrology/"
ValidationRunName = 'Hydrology-CamelsUS-anushka'

# Define the file name and file path
timeseries_file = 'BasicInputTimeSeries.npy'
static_file = 'BasicInputStaticProps.npy'
metadata_file = 'metadata.json'

timeseries_data = np.load(APPLDIR + timeseries_file, allow_pickle=True)
static_data = np.load(APPLDIR + static_file, allow_pickle=True)

# Metadata
with open(APPLDIR + metadata_file, 'r') as f:
    metadata = json.load(f)

TrainingNloc, ValidationNloc, ListofTrainingLocs, ListofValidationLocs = restoreValLocs(ValidationRunName)
ListofTrainingLocs = [metadata['locs'][i] for i in ListofTrainingLocs]
ListofValidationLocs = [metadata['locs'][i] for i in ListofValidationLocs]

In [ ]:
# Prepare input
timeseries_cols = metadata['BasicInputTimeSeries']['fields']
static_cols = metadata['BasicInputStaticProps']['fields']

# Create the DataFrame
df_timeseries = pd.DataFrame(timeseries_data, columns=timeseries_cols)
df_static = pd.DataFrame(static_data, columns=static_cols)

df_timeseries['ds'] = pd.to_datetime(df_timeseries['Year_Mnth_Day'])
df_timeseries['unique_id'] = df_timeseries['basin_id']
df_static = df_static.rename(columns={"gauge_id": "basin_id"})
merged_df = pd.merge(df_timeseries, df_static, on='basin_id', how='inner')
merged_df.drop(['Year_Mnth_Day', 'basin_id'], axis=1, inplace=True)

df_timeseries.drop(['Year_Mnth_Day', 'basin_id'], axis=1, inplace=True)
scaler = MinMaxScaler(feature_range=(0, 1))

for col in df_timeseries.columns:
    if col != 'ds' and col !='unique_id':
        if col == 'prcp(mm/day)' or col == 'QObs(mm/d)':
            df_timeseries[col] = df_timeseries[col].clip(lower=0)
            df_timeseries[col] = df_timeseries[col]**(1/3)

        df_timeseries[col] = scaler.fit_transform(df_timeseries[[col]])

print(df_timeseries)

In [ ]:
# load pretrain model
# supports different lookback/forecast lengths
model = AutoModelForCausalLM.from_pretrained('thuml/sundial-base-128m', trust_remote_code=True)

# perpare input
lookback_length = 21
Nloc = 671
Ndays = 7031

# forecasting configurations
forecast_length = 1       # forecast the next 288 timestamps
num_samples = 1           # generate 1 sample

timeseries_cols = timeseries_cols[2:]
Nprop = len(timeseries_cols)

In [ ]:
# evaluate model
ground_truth = np.zeros((Ndays - lookback_length, ValidationNloc, Nprop))
preds = np.zeros((Ndays - lookback_length, ValidationNloc, Nprop))
iloc = 0

if Restorefromcheckpoint:
    iloc, ground_truth, preds = restoreCkpt(run_name, ckpt_name)
    iloc += 1   # start evaluating the next unchecked location
    print(f"Restored from checkpoint: iloc={iloc}")


# for each input time series property
iprop = timeseries_cols.index(timeseries_prop)
print(f"Starting inferencing on prop {iprop}: {timeseries_prop}")
print("="*100)

# prepare input
df = (df_timeseries.set_index(['ds','unique_id']).unstack('unique_id')[timeseries_prop].sort_index())

# look at each catchment
while iloc < ValidationNloc:
    cat_id = ListofValidationLocs[iloc]
    # look at each sequence for that catchment
    for t in range(Ndays - lookback_length):
        normalized_input = df[cat_id][t: t + lookback_length]
        lookback = torch.tensor(df[cat_id][t: t + lookback_length]).unsqueeze(0).float()
        forecast = model.generate(lookback, max_new_tokens=forecast_length, num_samples=num_samples)
        # update preds & label
        ground_truth[t, iloc, iprop] = df[cat_id][t + lookback_length]
        preds[t, iloc, iprop] = forecast[0][0].float()

    saveCkpt(iloc, ground_truth, preds, run_name, ckpt_name)

    iloc += 1
    print(f"Location {iloc} saved to checkpoint")


In [ ]:
# calculate MSE
for iprop in range(len(timeseries_cols)):
    prop = timeseries_cols[iprop]
    mse = np.mean((ground_truth[:, :, iprop] - preds[:, :, iprop])**2)
    rmse = np.sqrt(mse)
    print(f"MSE for {prop}: {mse}")
    print(f"RMSE for {prop}: {rmse}")

In [ ]:
# plot
fig, axs = plot_numpy(ground_truth[:, :, iprop], pred[:, :, iprop], f'Sundial {timeseries_prop}', splitsize=6)